In [26]:
from openai import OpenAI
from key import openai_key
client = OpenAI(api_key=openai_key)

batch_input_file = client.files.create(
  file=open("../../batch_request.jsonl", "rb"),
  purpose="batch"
)

In [27]:
batch_input_file_id = batch_input_file.id

metadata = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "version 1.0",
    }
)

metadata

Batch(id='batch_67280b3b035881908257f3c273e4354b', completion_window='24h', created_at=1730677563, endpoint='/v1/chat/completions', input_file_id='file-BasbSBsY7xwzkz9LzAZ81u3p', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1730763963, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'test'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

# Wait until it finished, you can run it if you are not sure whether it is finished

In [33]:
batch_id = metadata.id
metadata = client.batches.retrieve(batch_id)
output_file_id = metadata.output_file_id

file_response = client.files.content(output_file_id)
output_file_id
# print(file_response.text)

{"id": "batch_req_67280bb1e96c81908cf5fc17b0dd8226", "custom_id": "10002yt", "response": {"status_code": 200, "request_id": "979e96804263834879a1add7c1ab558b", "body": {"id": "chatcmpl-APebK8Thb4XMMMWHVapmkg7LkenZ4", "object": "chat.completion", "created": 1730677634, "model": "gpt-4o-2024-08-06", "choices": [{"index": 0, "message": {"role": "assistant", "content": "(1) relation: no, related phrases if any: \n(2) protein: no, related phrases if any: \n(3) ed: no, related phrases if any: \n(4) exercise: yes, related phrases if any: \"cardio\"\n(5) meal: no, related phrases if any: \n(6) crave: no, related phrases if any: \n(7) restrict: no, related phrases if any: \n(8) binge: no, related phrases if any: \n(9) loss: yes, related phrases if any: \"20 pound down\"\n(10) gain: no, related phrases if any: \n(11) calorie: no, related phrases if any: \n(12) thinspo: no, related phrases if any: \n(13) leanbody: yes, related phrases if any: \"wellness show\"\n(14) bodyhate: no, related phrases 

In [55]:
ids = []
responses = []
for data in file_response.iter_lines():
    # print(data)
    output = json.loads(data)
    
    # print(output['custom_id'], output['response']['body']['choices'][0]['message']['content'])
    ids.append(output['custom_id'])
    responses.append(output['response']['body']['choices'][0]['message']['content'])

import pandas as pd
df = pd.DataFrame({"sm_id": ids,"responses": responses})

# Write to CSV while preserving line breaks
csv_file_path = "output.csv"
df.to_csv(csv_file_path, index=False, quoting=1)  # quoting=1 is equivalent to csv.QUOTE_ALL

print(f"CSV file created at {csv_file_path}")


CSV file created at output.csv


In [56]:
read_df = pd.read_csv(csv_file_path)

In [58]:
read_df.head()

,sm_id,responses
0,10002yt,"(1) relation: no, related phrases if any: \n(2..."
1,1003i3b,"(1) relation: yes, related phrases if any: ""al..."


In [ ]:
read